In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

In [2]:
config = {
    "learning_rate": 2e-5,
    "epoch": 6,
    "batch_size": 32
}

CFG = SimpleNamespace(**config)

In [3]:
train_df = pd.read_csv("C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/train_df_1012.csv")
test_df = pd.read_csv("C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/test_df_1012.csv")

In [4]:
# # 토크나이저와 모델 변경
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertTokenizer, RobertaForSequenceClassification

In [7]:
import random
import time
import datetime
import gc
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModelForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup,DistilBertTokenizer

In [35]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=len(train_df['분류'].unique())).to(device)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
# Define helper function to tokenize the dataset
def tokenize_data(example):
    return tokenizer(
        example['키워드'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

In [31]:
train_texts = train_df[['키워드']].values.flatten()
train_labels = train_df['분류'].astype('category').cat.codes.values

In [32]:
# Split into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Tokenize the texts
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

In [33]:
# Convert to dataset format
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [34]:
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

In [40]:
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save training outputs
    evaluation_strategy="epoch",    # When to run evaluations
    per_device_train_batch_size=64,  # Batch size for training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    num_train_epochs=3,             # Number of training epochs
    logging_dir="./logs",           # Directory for storing logs
    logging_steps=10,               # How often to log
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()